<a href="https://colab.research.google.com/github/bootneck2000/Yolo_FCNN_dev_pengos/blob/main/Faster_RCNN_train_pengo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 1: Install required packages (if not already installed)
#!pip install torch torchvision requests opencv-python tqdm pillow
#!pip install --upgrade numpy==1.26.4
# Cell 2: Imports
import os
import requests
import zipfile
import io
import shutil
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.transforms import functional as F
import numpy as np
from tqdm import tqdm

# Cell 3: Utility to download and extract Roboflow YOLOv8 export
def download_and_extract_roboflow_zip(roboflow_zip_url, extract_to="roboflow_dataset"):
    if os.path.exists(extract_to):
        print(f"'{extract_to}' already exists, skipping download.")
        return
    print("Downloading dataset from Roboflow...")
    r = requests.get(roboflow_zip_url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(extract_to)
    print(f"Dataset extracted to '{extract_to}'.")

# Cell 4: YOLOv8 to COCO box conversion (helper)
def yolo_to_coco(bbox, img_w, img_h):
    # bbox: [x_center, y_center, width, height] normalized
    x_c, y_c, w, h = bbox
    x_c *= img_w
    y_c *= img_h
    w *= img_w
    h *= img_h
    # Convert to [xmin, ymin, xmax, ymax]
    xmin = x_c - w / 2
    ymin = y_c - h / 2
    xmax = x_c + w / 2
    ymax = y_c + h / 2
    return [xmin, ymin, xmax, ymax]

# Cell 5: Roboflow YOLO8 Dataset
class RoboflowYolo8Dataset(Dataset):
    def __init__(self, data_dir, split='train', transforms=None):
        self.data_dir = data_dir
        self.split = split
        self.transforms = transforms

        # Find image and label files
        self.images_dir = os.path.join(self.data_dir, self.split, "images")
        self.labels_dir = os.path.join(self.data_dir, self.split, "labels")
        self.image_files = [f for f in os.listdir(self.images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        self.label_files = [f.replace(".jpg", ".txt").replace(".png", ".txt").replace(".jpeg",".txt") for f in self.image_files]

        # Build class list from data.yaml
        self.class_dict = self._parse_class_names()

    def _parse_class_names(self):
        yaml_path = os.path.join(self.data_dir, "data.yaml")
        import yaml
        with open(yaml_path, "r") as f:
            ydata = yaml.safe_load(f)
        names = ydata.get("names") or ydata.get("nc")
        if isinstance(names, dict):
            names = [names[k] for k in sorted(names)]
        return {name: idx for idx, name in enumerate(names)}

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        label_name = self.label_files[idx]
        img_path = os.path.join(self.images_dir, img_name)
        label_path = os.path.join(self.labels_dir, label_name)

        # Load image
        image = Image.open(img_path).convert("RGB")
        img_w, img_h = image.size

        # Read label file (YOLO format)
        boxes = []
        labels = []
        if os.path.exists(label_path):
            with open(label_path, "r") as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) < 5: continue
                    cls_id = int(parts[0])
                    bbox = list(map(float, parts[1:5]))
                    box = yolo_to_coco(bbox, img_w, img_h)
                    boxes.append(box)
                    labels.append(cls_id+1)  # +1 for background=0 in FasterRCNN

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        target = {
            "boxes": boxes,
            "labels": labels,
            "image_id": torch.tensor([idx]),
        }
        if self.transforms:
            image = self.transforms(image)
        else:
            image = F.to_tensor(image)
        return image, target

# Cell 6: Helper for collate_fn (required by DataLoader)
def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
# Cell 7: Prepare the dataset
# Set your Roboflow ZIP URL here (YOLOv8 format)
ROBOFLOW_ZIP_URL = "https://app.roboflow.com/ds/VquXPr9PVz?key=QyrlUzVsjR"
DATASET_DIR = "roboflow_dataset"
download_and_extract_roboflow_zip(ROBOFLOW_ZIP_URL, extract_to=DATASET_DIR)

# Prepare train/val datasets and dataloaders
train_dataset = RoboflowYolo8Dataset(DATASET_DIR, split="train")
#val_dataset = RoboflowYolo8Dataset(DATASET_DIR, split="valid")
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)
#val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False, collate_fn=collate_fn)

In [ ]:
# Cell 8: Prepare the model
def get_fasterrcnn_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

num_classes = len(train_dataset.class_dict) + 1  # +1 for background
model = get_fasterrcnn_model(num_classes)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Cell 9: Training loop
num_epochs = 10
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        running_loss += losses.item()
    print(f"Epoch {epoch+1} - Average Loss: {running_loss/len(train_loader):.4f}")

# Cell 10: Save the trained model
torch.save(model.state_dict(), "fasterrcnn_roboflow_yolov8.pth")
print("Model saved as fasterrcnn_roboflow_yolov8.pth")

In [ ]:
import os
for root, dirs, files in os.walk("roboflow_dataset"):
    print(root)

In [ ]:
import numpy as np
print(np.__version__)


In [ ]:
pwd

'/Users/andylowther'